# QOSF 10
<h2>Task 3 - Optimization of the 2D Bin Packing Problem</h2>

<figure>
  <center>
  <img src="img/bpp.png" alt="My image caption" height=250>
  <figcaption>Instance of 2D BPP solved using quantum annealing</figcaption>
  </center>
</figure>

### Project Summary

Imagine you're packing for a flight — whether you realize it or not, you're tackling a classic optimization challenge known as the **Bin Packing Problem (BPP)**. In its three-dimensional form, the task is to efficiently fit as many items (cases) as possible into a limited number of suitcases (bins), while also adhering to weight restrictions imposed by the airline. By considering these real-world constraints, BPP can be divided into two subproblems:
- <u>Weight</u>: By choosing whether or not to include a case in a particular bin, stay under the declared weight limit per bin
- <u>Space</u>: Assuming all the cases can fit by rearranging them between bins, find a spatial configuration that manages to fit all the cases

The weight problem is formulated as a constrained optimization over binary variables, see [[1]](#openqaoa). In this work we focus on optimizing for space, deriving mainly from [[2]](#dwave3d), and break down the problem by simplifying it to two dimensions.

The 2D BPP is described mathematically as a constrained quadratic model (CQM). We describe the coordinates of case $i$ as $(x_i, y_i)$. These are real-valued variables which pose a challenge for implementing on a digital quantum computer. Fortunately a solution can be found on D-Wave's `LeapHybridCQMSampler` backend which is a quantum-classical hybrid sampler [[3]](#ocean) that uses GPUs to handle the non-binary continuous variables in the CQM.

First, import the required packages.

In [1]:
import pickle
import dimod
import packing2d as bpp
import pennylane as qml
import pennylane.numpy as np

### Problem instance and constraints
We can use a stored solution from a past run, or start a job to run on the cloud (requires sign-in to Leap).

In the original implementation from [[2]](#dwave3d), the user has the ability to generate random problem instances in a slider-based GUI. Our port reads in a 2D BPP instance from a file, see `packing2d.py` for more information. This is done on purpose to stay within the free tier limits of the cloud service. As an example we choose medium-seized instance of BPP (see `test_data.txt`) which requires 2 bins and runs relatively quick on the cloud. 

In [2]:
# Run on D-Wave service (True), or use a cached run (False)
QPU = False

cached = {
    "med" : {"p" : "cache/test_data_1.txt", "s" : "cache/solution_1.pickle"},
    "easy" : {"p" : "cache/test_data_2.txt", "s" : "cache/solution_2.pickle"}
}

data = bpp.read_instance(cached["med"]["p"])

The following section is a description of the CQM:
- the variables pertaining to BPP
- an expression to be extremized (objective)
- a list of equalities and inequalities (constraints) that must be satisfied by a feasible solution on our problem variables.

<hr>

#### Objective
The objective for our CQM is three-fold:
- Minimize average height of cases:
    $$
    o_1 = \frac{\sum_i y_i + y_i^\prime}{m}
    $$
    where $y_i^\prime$ is the effective height (more later) and $m$ is the number of cases.
- Minimize height of the case at the top of the bin:
    $$
    o_2 = \sum_j s_j
    $$
    where $s_j$ is the height of the highest case in bin $j$.
- Minimize the number of used bins:
    $$
    o_3 = H\sum_j v_j
    $$
    where $v_j = 1$ if it contains atleast one case, and $H$ is the height of the bins. All bins have the same height which simplifies the problem, but not all bins are treated equally (more later).

These objectives turn out to the be the same no matter the dimension of BPP, and the total objective is the sum $o_1+o_2+o_3$.


#### Orientation constraints
<figure>
  <center>
  <img src="img/orientation.svg" alt="My image caption" height=250>
  <figcaption>Two possible orientations of the same case</figcaption>
  </center>
</figure>
Each case has only one orientation, which is enforced by:

$$
\sum_{i,k} r_{i,k} = 1
$$

where index $k$ runs over the two orientations in 2D shown above. In 3D, this would be 6 orientations. 

**Sidenote:** In a general axis-aligned BPP in $n$ dimensions we assign each axis of the hyperrectangle to one axis of the bin, which is $n!$ orientations, or the number of permutations of $n$ items without repetition.

This is where the effective width and height of a case comes in handy. It chooses an orientation from these allowed permutations:

$$
\begin{aligned}
x_i^\prime = w_i r_{i,1}+ h_i r_{i,2}\\
y_i^\prime = h_i r_{i,1}+ w_i r_{i,2}
\end{aligned}
$$


#### Case and bin assignment constraints
We calculate up front based on the total area of all the cases how many bins we actually need. Then we manually place the first case in the first bin. This leads to a series of constraints:
- Each case goes to exactly one bin:
    $$
    \sum_j u_{i,j} = 1
    $$
- Each case goes to exactly one bin:
    $$
    \sum_i (1-v_j)u_{i,j} \le 0
    $$
- Ensure bins are added in order:
    $$
    v_{j+1} - v_j \le 0 \qquad j \ne n
    $$


#### Geometric constraints
<figure>
  <center>
  <img src="img/geometric.svg" alt="My image caption" height=250>
  <figcaption>Four possible placements of one case, relative to another</figcaption>
  </center>
</figure>

We define the binary variable $b_{i,k,q}$ which defines the geometric relation $q \in \{0,1,2,3\}$ between cases $i$ and $k$. As $q$ is unsigned, it seems to be cleverly optimized behind the scenes to $q \in \{00, 01, 10, 11\}$. This means the following geometric constraints do not involve integer variables, which we notice in the CQM stats later. So the binary constraint indexed by $q$ encodes the four possible placements in 2D:
1. Case $i$ is to the left of case $k$ ($q=0$)
$$
-(2-u_{i,j}u_{k,j}-b_{i,k,0})nW + (x_i+x_i^\prime-x_k) \le 0
$$
2. Case $i$ is above case $k$ ($q=1$)
$$
-(2-u_{i,j}u_{k,j}-b_{i,k,1})H + (y_i+y_i^\prime-y_k) \le 0
$$
3. Case $i$ is to the right of case $k$ ($q=2$)
$$
-(2-u_{i,j}u_{k,j}-b_{i,k,2})nW + (x_k+x_k^\prime-x_i) \le 0
$$
4. Case $i$ is below case $k$ ($q=3$)
$$
-(2-u_{i,j}u_{k,j}-b_{i,k,3})H + (y_k+y_k^\prime-y_i) \le 0
$$
We have to multiply the width $W$ by number of bins $n$ because we stack the bins left to right. $x_i = 0$ means bottom left of case $i$ is located at the bottom left of the first bin. 

To enforce only one placement of case $i$ relative to case $k$, we have:
$$
\sum_{i,k,q} b_{i,k,q} = 1
$$

**Sidenote:** A BPP in $n$ dimensions would have $2n$ geometric relations, one for each face of the hyperrectangle.

#### Bin boundary constraints
These sets of constraints ensure that case $i$ is not placed outside the bins
$$
\begin{aligned}
y_i + y_i^\prime - H \le 0\\
y_i + y_i^\prime - s_j - (1-u_{i,j})nW \le 0\\
x_i + x_i^\prime - W(j+1) - (1-u_{i,j})nW \le 0\\
jWu_{i,j} - x_i \le 0
\end{aligned}
$$

This concludes our mathematical description of the CQM.
<hr>

Notice that we can frame every inequality constraint as less-than-equal (LE) to zero. According to [[4]](#dwavewp), the entire CQM can be cast as a minimization of the following:
$$
\argmin \quad \text{objective} + \text{square of the LHS of LE constraint}
$$
If one tries to manually change any of the LE constraints to GE, we can see this reflected in the CQM stats. But `dimod` would need to convert them back to LE under the hood to perform the above minimization.

We now set up the CQM as stated above by specifying our variables (binary or real), and objectives and constraints on the variables.

In [3]:
cases = bpp.Cases(data)
bins = bpp.Bins(data, cases)
vars = bpp.Variables(cases, bins)

cqm, effective_dimensions = bpp.build_cqm(vars, bins, cases)

Number of cases: 23
Minimum Number of bins required: 2


A summary of the CQM (stats) is shown below. Some observations:
- Clearly the nubmer of variables grows with the number of cases and bins.
- As mentioned earlier, the CQM constraints were all described as LT, so GT is zero. If we flip any constraints, LT + GT should stay constant if the problem size is fixed, i.e, we fix the number of cases and bins.
- If we restrict to a much simpler problem of only one bin, we see that all the quadratic constraints vanish. 

In [4]:
bpp.print_cqm_stats(cqm)

 
===================================MODEL INFORMATION===================================
          Variables                    Constraints               Sensitivity
------------------------------ ---------------------------- ------------------
  Binary    Integer    Continuous    Quad    Linear    One-hot    EQ      LT    GT
--------  ---------  ------------  ------  --------  ---------  ------  ----  ----
    1102          0            48    1848       612        298     298  2162     0


We note that the CQM objective is not an equation but an expression that needs to be minimized. 

In [5]:
print(cqm.objective.to_polystring())

0.043478260869565216*y_0 + 0.34782608695652173*o_0_0 + 0.21739130434782608*o_0_1 + 0.043478260869565216*y_1 + 0.34782608695652173*o_1_0 + 0.21739130434782608*o_1_1 + 0.043478260869565216*y_2 + 0.5217391304347826*o_2_0 + 0.5217391304347826*o_2_1 + 0.043478260869565216*y_3 + 0.5217391304347826*o_3_0 + 0.5217391304347826*o_3_1 + 0.043478260869565216*y_4 + 0.5217391304347826*o_4_0 + 0.5217391304347826*o_4_1 + 0.043478260869565216*y_5 + 0.5217391304347826*o_5_0 + 0.5217391304347826*o_5_1 + 0.043478260869565216*y_6 + 0.5217391304347826*o_6_0 + 0.5217391304347826*o_6_1 + 0.043478260869565216*y_7 + 0.5217391304347826*o_7_0 + 0.5217391304347826*o_7_1 + 0.043478260869565216*y_8 + 0.5217391304347826*o_8_0 + 0.5217391304347826*o_8_1 + 0.043478260869565216*y_9 + 0.5217391304347826*o_9_0 + 0.5217391304347826*o_9_1 + 0.043478260869565216*y_10 + 0.4782608695652174*o_10_0 + 0.34782608695652173*o_10_1 + 0.043478260869565216*y_11 + 0.4782608695652174*o_11_0 + 0.34782608695652173*o_11_1 + 0.04347826086956

## Quantum annealer solution

Now we try to solve our CQM by calling a sampler. Note that this process is non-deterministic. We may get a different result each time we run so a global optimum is not guaranteed. Depending on the problem and the solver used  we can pass in some arguments to get more samples out per run, see [[3]](#ocean).

In [6]:
best_feasible = None

if QPU:
    best_feasible = bpp.call_solver(cqm, 20)
else:
    # Deserialize saved solution
    with open(cached["med"]["s"], "rb") as infile:
        best_feasible = pickle.load(infile)
        infile.close()


Let's visualize our solution:

In [7]:
fig = bpp.plot_rects(
    best_feasible, vars, cases, bins, effective_dimensions
)
fig.show()

Hovering over the center of a case or the text below a bin will show some details of the case/bin. The plot is rescaled automatically based on the problem size.

## VQA and QAOA

Our chosen flavor of BPP (optimizing space) has many variables, many of them continuous and real-valued. Encoding continuous data into qubits is possible ([`qml.AngleEmbedding`](https://docs.pennylane.ai/en/stable/code/api/pennylane.AngleEmbedding.html)) but reading it out is challenging. In this section we simplify the BPP instance greatly, and describe a possible strategy to solve the problem on a large enough quantum computer. Note that while running this problem is impractical right now, we provide some code that show a possible approach to tackle this problem.

We need to discretize space into a lattice, that looks something like this:

<figure>
  <center>
  <img src="img/partition.svg" alt="My image caption" height=250>
  <figcaption>Discretizing 2D space using two qubits for each coordinate</figcaption>
  </center>
</figure>

The following code sets up some useful parameters:

In [8]:
# Number of qubits needed to represent a floating point value in [0,1)
BITS_PRECISION = 8

# Number of layers in QAOA
NUM_LAYERS = 5

Here we use a much smaller problem instance of 4 cases that fit exactly into 2 bins. But even with a reduced problem size, we require ~100 qubits.

In [9]:
data = bpp.read_instance(cached["easy"]["p"])

cases = bpp.Cases(data)
bins = bpp.Bins(data, cases)
vars = bpp.Variables(cases, bins)

cqm, effective_dimensions = bpp.build_cqm(vars, bins, cases)

bpp.print_cqm_stats(cqm)

Number of cases: 4
Minimum Number of bins required: 2
 
===================================MODEL INFORMATION===================================
          Variables                    Constraints               Sensitivity
------------------------------ ---------------------------- ------------------
  Binary    Integer    Continuous    Quad    Linear    One-hot    EQ      LT    GT
--------  ---------  ------------  ------  --------  ---------  ------  ----  ----
      38          0            10      24        61         13      13    72     0


The small BPP instance admits a tight fit, as per the annealing solution:

In [10]:
best_feasible = None

if QPU:
    best_feasible = bpp.call_solver(cqm, 20)
else:
    # Deserialize saved solution
    with open(cached["easy"]["s"], "rb") as infile:
        best_feasible = pickle.load(infile)
        infile.close()

fig = bpp.plot_rects(
    best_feasible, vars, cases, bins, effective_dimensions
)
fig.show()

The number of qubits is shown here, after we map the CQM variables to quantum wires. We allocate a quantum register of size `BITS_PRECISION` to each continuous variable. Reducing this value will lower the number of qubits, at the cost of a significant impact on the accuracy of our solution. Reducing `BITS_PRECISION` may also make constraints harder to satisfy. In any case, since we are not running this code we will press on.

In [11]:
assert(BITS_PRECISION > 1)

# Mapping variables in CQM to tensor product of Pauli Z operators on wires
wire_idx = 0
wire_map = {}
for v in list(cqm.variables):
    if cqm.vartype(v) is dimod.BINARY:
        wire_map[v] = qml.Z(wire_idx)
        wire_idx += 1
    elif cqm.vartype(v) is dimod.REAL:
        op = qml.Z(wire_idx)
        for i in range(1, BITS_PRECISION):
            op = op @ qml.Z(wire_idx + i)
        wire_map[v] = op
        wire_idx += BITS_PRECISION

print(f"Number of qubits required: {wire_idx}")
print(f"Mapping CQM variables to quantum wires:\n{wire_map}")

Number of qubits required: 118
Mapping CQM variables to quantum wires:
{'o_0_0': Z(0), 'o_0_1': Z(1), 'o_1_0': Z(2), 'o_1_1': Z(3), 'o_2_0': Z(4), 'o_2_1': Z(5), 'o_3_0': Z(6), 'o_3_1': Z(7), 'case_1_in_bin_0': Z(8), 'case_1_in_bin_1': Z(9), 'case_2_in_bin_0': Z(10), 'case_2_in_bin_1': Z(11), 'case_3_in_bin_0': Z(12), 'case_3_in_bin_1': Z(13), 'sel_0_1_0': Z(14), 'sel_0_1_1': Z(15), 'sel_0_1_2': Z(16), 'sel_0_1_3': Z(17), 'sel_0_2_0': Z(18), 'sel_0_2_1': Z(19), 'sel_0_2_2': Z(20), 'sel_0_2_3': Z(21), 'sel_0_3_0': Z(22), 'sel_0_3_1': Z(23), 'sel_0_3_2': Z(24), 'sel_0_3_3': Z(25), 'sel_1_2_0': Z(26), 'sel_1_2_1': Z(27), 'sel_1_2_2': Z(28), 'sel_1_2_3': Z(29), 'sel_1_3_0': Z(30), 'sel_1_3_1': Z(31), 'sel_1_3_2': Z(32), 'sel_1_3_3': Z(33), 'sel_2_3_0': Z(34), 'sel_2_3_1': Z(35), 'sel_2_3_2': Z(36), 'sel_2_3_3': Z(37), 'x_0': Z(38) @ Z(39) @ Z(40) @ Z(41) @ Z(42) @ Z(43) @ Z(44) @ Z(45), 'x_1': Z(46) @ Z(47) @ Z(48) @ Z(49) @ Z(50) @ Z(51) @ Z(52) @ Z(53), 'y_0': Z(54) @ Z(55) @ Z(56) @ Z(5

Here we attempt to parse the CQM into a quantum Hamiltonian. The following code is included for completeness, and is meant simply as an illustration of the kind of quantum circuit that is produced. It likely needs many significant changes, like how the LE constraints are handled, and actually using the constant term of the CQM (`const_term`) in our objective being minimized.

In [12]:
coeffs = []
ops = []
const_term = 0

def is_float(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def eqn_parser(eqn_str, coeffs, ops, const_term, is_obj=False):
    '''
    Implements an equation parser that combines both the objective and constraints
    in the ``dimod.CQM``.
    
    Args:
        eqn_str (string): An equation in the BQM
        coeffs (list[float]): The coefficients array of the mixer Hamiltonian
        ops (list[qml.Operation]): The array tensor product terms in the mixer Hamiltonian
        const_term (float): Constant term in the overall cost function
        is_obj (bool): 
    '''
    # If eqn is an objective function, append RHS and proceed as usual
    if is_obj:
        eqn_str += " <= 0"
    
    parsed = str(eqn_str).split(" ")
    lhs = parsed[:-2]
    rhs = parsed[-2:]
    
    # Equality constraint (one hot)
    if rhs[0] == "==":
        const_term += -1
    
    for i in range(len(lhs)):
        # Parse constant
        if is_float(lhs[i]):
            const_term += float(lhs[i])
        # Parse term
        elif lhs[i] != "-" and lhs[i] != "+":
            coeff = 1
            if i > 0 and lhs[i-1] == "-": # Subtracted term
                coeff = -1
            if lhs[i][0] == '-':
                lhs[i] = lhs[i][1:] # First term is negative
                coeff = -1

            tensor_terms = []
            for term in lhs[i].split("*"):
                if term in wire_map.keys():
                    tensor_terms.append(wire_map[term])
                elif is_float(term):
                    coeff *= float(term)
            op = tensor_terms[0]
            for j in range(1, len(tensor_terms)):
                op = op @ tensor_terms[j]
            
            coeffs.append(coeff)
            ops.append(op)

# Parse CQM objective and constraints
eqn_parser(cqm.objective.to_polystring(), coeffs, ops, const_term, True)
for v in cqm.constraints.values():
    eqn_parser(v, coeffs, ops, const_term)

A quantum device is created with the required number of wires (not intended to run).

In [13]:
dev = qml.device("default.qubit", wires=wire_idx)

The reason to use `eqn_parser` was to obtain a cost Hamiltonian $H_C$, useful to optimize the VQA. For QAOA, we use the standard mixer Hamiltonian $H_M$ (the same as `qml.qaoa.mixers.x_mixer`), a simple non-commuting sum of Pauli-X operations on each qubit. It is not clear whether this mixer is beneficial for BPP.

In [14]:
cost_h = qml.Hamiltonian(coeffs, ops)
mixer_h = qml.Hamiltonian([1] * wire_idx, [qml.PauliX(i) for i in range(wire_idx)])


For a VQA ansatz we may use `qml.StronglyEntanglingLayers`, as well as the choice of either analytic gradients or gradients derived from numerical differentiation. These are used by a classical optimizer like `qml.AdamOptimizer` that sample the quantum circuit and update the parameters in a hybrid loop.

In [15]:

@qml.qnode(dev)
def strongly_entangling_ansatz(observable,params):
    """Applies an ansatz with moderate entanglement.

    Args:
        observable (qml.op): a pennylane operator whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz. They have the shape of `qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_bits)`

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of the given observable.
    """
    qml.StronglyEntanglingLayers(weights=params, wires=range(wire_idx), ranges=[wire_idx-1])
    return qml.expval(observable)

def optimizer(observable,params):
    """Updates the parameters to minimize the cost function value.

    Args:
        observable (qml.Hamiltonian): a pennylane Hamiltonian whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz.

    Returns:
        (np.array): an array with the optimized trainable parameters.
    """
    opt = qml.AdamOptimizer()
    for _ in range(1000):
        _, params = opt.step(strongly_entangling_ansatz, observable, params)
    return params 

QAOA is simply another ansatz that is useful in VQA. The following code is the quantum circuit for QAOA that uses built-in layers in `qml.qaoa.layers` which consist of repeated application of the cost and mixer layers.

In [16]:
assert(NUM_LAYERS >= 1)
params=np.ones((NUM_LAYERS, 2),requires_grad=True) * 0.5

def qaoa_layer(params,cost_h):
    """Implement one QAOA layer alternating H_C and H_M.

    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        cost_h (qml.Hamiltonian): The cost Hamiltonian
    """  
    qml.qaoa.layers.cost_layer(params[0], cost_h)
    qml.qaoa.layers.mixer_layer(params[1], mixer_h)
    
def qaoa_circuit(params,L,cost_h):
    """Implement the initial state and p layers of the QAOA ansatz.

    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        p (int): Number of layers of the QAOA ansatz.
        cost_h (qml.Hamiltonian): The cost Hamiltonian
    """  
    qml.Hadamard(0)
    for i in range(L):
        qaoa_layer(params[i],cost_h)
    
@qml.qnode(dev)
def probability_circuit(params,cost_h):
    """QAOA circuit which returns the probabilities.
    Args:
        params (np.array): An array with the trainable parameters of the QAOA ansatz.
        p (int): Number of layers of the QAOA ansatz.
        cost_h (qml.Hamiltonian): The cost Hamiltonian
    Returns:
        (np.tensor): A tensor with the probabilities of measuring the quantum states.
    """  
    qaoa_circuit(params,NUM_LAYERS,cost_h)
    return qml.probs(wires=0)

## References

<a id="openqaoa"></a>
[1] OpenQAOA: https://openqaoa.entropicalabs.com/problems/bin-packing/

<a id="dwave3d"></a>
[2] 3D Bin Packing sample: https://github.dev/dwave-examples/3d-bin-packing

<a id="ocean"></a>
[3] Ocean Programs for Beginners: https://www.dwavesys.com/media/fmtj2fw3/20210920_ofbguide.pdf

<a id="dwavewp"></a>
[4] Problem Formulation Guide: https://www.dwavesys.com/media/bu0lh5ee/problem-formulation-guide-2022-01-10.pdf